In [3]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

import numpy as np
from sklearn import *
from lstm.imdb_lstm import *

import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


# Load Dataset
We reload the dataset with the plain text plots and the labels that reef generated

In [5]:
dataset='imdb'

from data.imdb_loader import DataLoader
# from data.youtube_loader import DataLoader
# from data.trec_loader import DataLoader
# from data.sms_loader import DataLoader

dl = DataLoader()
train_primitive_matrix, val_primitive_matrix, test_primitive_matrix, train_ground, val_ground, \
test_ground, _, _, _, train_text, val_text, test_text = dl.load_data(dataset=dataset, split_val = 0.1)
# _, _, _, train_ground, val_ground, test_ground, train_text, val_text, test_text = dl.load_data(dataset=dataset)
# _, _, _, train_ground, val_ground, test_ground, _,_,_,train_text, val_text, test_text= dl.load_data(dataset=dataset)

len(labels 1920


In [6]:
fname = 'LFs/'+dataset+'/val_5_dict_dt1/normal_reef.npy'
train_reef = np.load(fname)

# Train an LSTM Model
We now train a simple LSTM model with the labels generated by Reef. The following hyperparameter search is simplistic, and a more fine-tuned search and a more complex model can improve performance!

__Note that this takes ~1 hour to run on CPU__

In [7]:
train_reef

array([0.5       , 0.5       , 0.5       , ..., 0.5       , 0.10919017,
       0.5       ])

In [8]:
f1_all = []
pr_all = []
re_all = []
val_acc_all = []


bs_arr = [64]#,128,256]
n_epochs_arr = [25]#[5,10,25]

for bs in bs_arr:
    for n in n_epochs_arr:
        y_pred = lstm_simple(train_text, train_reef, val_text, val_ground, bs=bs, n=n)
#         print(y_pred)
        predictions = np.round(y_pred)
        
        val_acc_all.append(np.sum(predictions == val_ground)/float(np.shape(val_ground)[0]))
        f1_all.append(metrics.f1_score(val_ground, predictions,average="macro"))
        pr_all.append(metrics.precision_score(val_ground, predictions))
        re_all.append(metrics.recall_score(val_ground, predictions))

W0509 13:13:41.652681 140160707962688 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0509 13:13:41.669822 140160707962688 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0509 13:13:41.673306 140160707962688 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0509 13:13:42.214559 140160707962688 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0509 13:13:42.2

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           226240    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 279,541
Trainable params: 279,541
Non-trainable params: 0
_________________________________________________________________
None


W0509 13:13:42.861724 140160707962688 deprecation_wrapper.py:119] From /home/ayusham/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 1278 samples, validate on 142 samples
Epoch 1/25
1278/1278 [==============================] - 15s 11ms/step - loss: 0.6844 - acc: 0.0000e+00 - val_loss: 0.6879 - val_acc: 0.5282
Epoch 2/25
1278/1278 [==============================] - 14s 11ms/step - loss: 0.6761 - acc: 0.0000e+00 - val_loss: 0.6685 - val_acc: 0.5282
Epoch 3/25
1278/1278 [==============================] - 14s 11ms/step - loss: 0.6482 - acc: 0.0000e+00 - val_loss: 0.5922 - val_acc: 0.6408
Epoch 4/25
1278/1278 [==============================] - 14s 11ms/step - loss: 0.6275 - acc: 0.0000e+00 - val_loss: 0.5823 - val_acc: 0.6831
Epoch 5/25
1278/1278 [==============================] - 14s 11ms/step - loss: 0.6164 - acc: 0.0000e+00 - val_loss: 0.5685 - val_acc: 0.6831
Epoch 6/25
1278/1278 [==============================] - 13s 10ms/step - loss: 0.6113 - acc: 0.0000e+00 - val_loss: 0.5623 - val_acc: 0.6972
Epoch 7/25
1278/1278 [==============================] - 13s 10ms/step - loss: 0.6087 - acc: 0.0000e+00 - val_loss

### Validation Performance

In [9]:
ii,jj = np.unravel_index(np.argmax(f1_all), (1,1))
print('Best Batch Size: ', bs_arr[ii])
print('Best Epochs: ', n_epochs_arr[jj])

print('Validation F1 Score: ', max(f1_all))
print('Validation Best Pr: ', pr_all[np.argmax(f1_all)])
print('Validation Best Re: ', re_all[np.argmax(f1_all)])

Best Batch Size:  64
Best Epochs:  25
Validation F1 Score:  0.6756644518272426
Validation Best Pr:  0.7555555555555555
Validation Best Re:  0.5074626865671642


### Test Performance
We re-train the model with the best validation performance since we don't save weights for the models currently.

In [14]:
y_pred = lstm_simple(train_text, train_reef, test_text, test_ground, bs=bs_arr[ii], n=n_epochs_arr[jj])
predictions = np.round(y_pred)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 32)           256288    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 101       
Total params: 309,589
Trainable params: 309,589
Non-trainable params: 0
_________________________________________________________________
None
Train on 1278 samples, validate on 500 samples
Epoch 1/25
1278/1278 [==============================] - 16s 12ms/step - loss: 0.6856 - acc: 0.0000e+00 - val_loss: 0.6907 - val_acc: 0.5260
Epoch 2/25
1278/1278 [==============================] - 14s 11ms/step - loss: 0.6782 - acc: 0.0000e+00 - val_loss: 0.6813 - val_acc: 0.5260
Epoch 3/25
1278/1278 [==============================] - 14s 11ms/ste

In [16]:
print ('Test F1 Score: ', metrics.f1_score(test_ground, predictions, average='macro'))
print ('Test Precision: ', metrics.precision_score(test_ground, predictions))
print ('Test Recall: ', metrics.recall_score(test_ground, predictions))

Test F1 Score:  0.6231974147510762
Test Precision:  0.7142857142857143
Test Recall:  0.4219409282700422


## [Optional] Ground Truth Performance
We can also train the same model with ground truth labels for the train set to see how far Reef labels are from the best possible performance.

In [12]:
y_pred = lstm_simple(train_text, train_ground, test_text, test_ground, bs=128, n=10)
predictions = np.round(y_pred)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 32)           256288    
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 309,589
Trainable params: 309,589
Non-trainable params: 0
_________________________________________________________________
None
Train on 1278 samples, validate on 500 samples
Epoch 1/10
1278/1278 [==============================] - 9s 7ms/step - loss: 0.6896 - acc: 0.5383 - val_loss: 0.6889 - val_acc: 0.5260
Epoch 2/10
1278/1278 [==============================] - 8s 6ms/step - loss: 0.6761 - acc: 0.5415 - val_loss: 0.6758 - val_acc: 0.5260
Epoch 3/10
1278/1278 [==============================] - 8s 6ms/step - loss: 0.64

### Test Performance

In [13]:
print ('Test F1 Score: ', metrics.f1_score(test_ground, predictions, average='macro'))
print ('Test Precision: ', metrics.precision_score(test_ground, predictions))
print ('Test Recall: ', metrics.recall_score(test_ground, predictions))

Test F1 Score:  0.6461397058823529
Test Precision:  1.0
Test Recall:  0.350210970464135
